## Trabajo práctico final | Ignacio Titimoli

## Conversión de archivos .pdf a .txt

Comenzaremos por importar las librerías requeridas.

In [1]:
import os
import PyPDF2

A continuación, setearemos el directorio de trabajo desde el cual tomaremos los archivos que necesitamos convertir a texto.

El código "loopea" entre todos los archivos .pdf guardados en el directorio predefinido.
Dentro del loop, el archivo PDF es aperturado en formato binario, utilizando la función `open`.

Utilizaremos `PyPDF2.PdfReader` para leer los archivos PDF, y el contenido será guardado dentro de la variable `pdf_reader`.
La función `extract_text()` es utilizada para que, en cada página del documento, extraiga el texto que encuentre.
Lo extraído se inserta dentro de la variable `text`.

A continuación, se crea un nuevo archivo de texto con el mismo nombre que figura en el pdf mediante `os.path.splitext()`, que parte el nombre del archivo y su extensión.

Finalmente, el texto extraído es escrito en el nuevo archivo de texto a partir de la función `write()`.
Luego, tanto el archivo PDF como el archivo de texto, son cerrados.

[Fuente](https://medium.com/mlearning-ai/extracting-text-from-multiple-pdf-files-with-python-and-pypdf2-b37f08ef728d)

In [2]:
pdf_dir = os.getcwd()

for filename in os.listdir(pdf_dir):
    if filename.endswith('.pdf'):
        pdf_file = open(os.path.join(pdf_dir, filename), 'rb')
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ''

        for i in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[i]
            text += page.extract_text()

        txt_filename = os.path.splitext(filename)[0] + '.txt'
        txt_file = open(os.path.join(pdf_dir, txt_filename), 'w')

        txt_file.write(text)

        pdf_file.close()
        txt_file.close()


A los efectos de poder realizar un taggeo de los datos en forma más rápida (aunque menos precisa), trabajeremos con una muestra del .txt obtenido ("BO-5350_2.txt"), que solo contiene una resolución dentro de la gran cantidad de resoluciones publicadas en el Boletín Oficial que utilizaremos como input.

## Implementación de modelo

Con ayuda de lo dispuesto en la siguiente [WEB](https://aws.amazon.com/es/blogs/machine-learning/developing-ner-models-with-amazon-sagemaker-ground-truth-and-amazon-comprehend/), implementaremos nuestro modelo para el etiquetado de entidades dentro de nuestra data.
La intención final es poder crear el siguiente flujo:

<img src="img/ner.jpg">

El proceso end-to-end consistirá en:
<ol>
    <li> Cargar el archivo (muestra de un Boletín Oficial) en S3 (punto 1).</li>
    <li> Utilizar el template mencionado previamente para generar la infraestructura necesaria en AWS (función Lambda y creación de buckets en S3) para llevar adelante este proceso. Se utilizará para lo dicho AWS CloudFormation. </li>
    <li> Crear un equipo de trabajo privado y utilizar NER dentro de Amazon SageMaker (Ground Truth) para etiquetar nuestra data (puntos 2 y 3).</li>
    <li> Realizar el trabajo de taggeo de nuestra data en Ground Truth de acuerdo con las entidades que son de nuestro interés (puntos 4a y b).</li>
    <li> Mediante el archivo manifiesto, transformar las etiquetas generadas sobre nuestra data input a un archivo en formato .csv para que estas puedan ser entrenadas dentro de Amazon Comprehend (puntos 5a y b).</li>
    <li> En Amazon Comprehend, lanzar un trabajo customizado de entrenamiento con NER, utilizando el dataset con entidades generado por la AWS Lambda (puntos 6a y b).</li>
    <li> Evaluar los resultados obtenidos.</li>

### Pipeline de conversión

Tomando como parámetro el siguiente [archivo YAML](https://aws-ml-blog.s3.amazonaws.com/artifacts/blog-groundtruth-comprehend-ner/cfn/cfn.yaml), iniciaremos dentro de CloudFormation el template que nos permitirá crear el bucket en S3, crear también la función Lambda y configurar la relación entre el bucket y dicha función para dispararla automáticamente cada vez que se detecte la llegada de archivos dentro de la carpeta `output.manifest`.

En nuestro caso, hemos creado el stack "ner-demo-lv" que, como mencionamos, convertirá a través de una función Lambda el archivo manifiesto aumentado generado en SageMaker Ground Truth a un formato reconocido por Amazon Comprehend para poder efectuar el entrenamiento.

<img src="img/cloud_formation.png">
<img src="img/cloud_formation2.png">
<img src="img/cloud_formation3.png">
<img src="img/cloud_formation4.png">

### Carga de archivos en S3

Cargaremos, dentro del bucket de S3 generado con CloudFormation, nuestra data cruda.

Como hemos mencionado, utilizaremos una muestra de nuestro Boletín Oficial ("BO_5350_2".txt), para poder trabajar con un corpus de texto más pequeño que nos ahorre tiempo al taggear las entidades.

<img src="img/s3.png">
<img src="img/s3_2.png">

### Ejecución del trabajo de etiquetado de entidades con NER

#### Crear un equipo de trabajo privado

Dentro de SageMaker, crearemos un equipo de trabajo privado que nos incluya a nosotros mismos. Este es el puntapié para poder, posteriormente, generar el trabajo de etiquetado y reconocimiento de entidades sobre nuestra data de input.

<img src="img/gt_equipoprivado.png">
<img src="img/gt_equipoprivado2.png">

#### Creación del archivo manifiesto

Generado el entorno de trabajo, utilizaremos el módulo de Ground Truth dentro de SageMaker para crear un trabajo de etiquetado.
Con este paso estaremos realizando:

<ol>
    <li> La creación del archivo manifiesto (que luego utilizaremos en formato json).</li>
    <li> Indicaremos desde dónde debe tomarse la data input. </li>
    <li> Indicaremos dónde deberá almacenarse la salida. </li>
    <li> Asignaremos el listado de entidades con el cual trabajeremos. </li>
</ol>

<img src="img/label_job_resume.png">

En el resumen de nuestro label job podemos ver:

- La ubicación en nuestro bucket de S3 del conjunto de datos de entrada.
- La ubicación en nuestro bucket de S3 del conjunto de datos de salida.

Por un error en la configuración, almacenamos la salida en la misma carpeta que la entrada (raw). Sin embargo, el archivo con los datos de entrada es el siguiente (aquí pueden verse las 186 sentencias):

<img src="img/label_job_entries.png">

La carpeta de salida ("etiqueta-boletin-3"), contiene a su vez cuatro subcarpetas. Una de ellas es "manifests", la cual, a su vez, dentro de la carpeta "output", contiene el archivo output.manifest.

<img src="img/label_job_output_folder.png">

Este es el archivo output.manifest.

<img src="img/output_manifest.png">

Y este es un ejemplo de una línea del archivo, en donde para el recurso "RESOLUCIÓN N.° 49/UPEJOL/18", se definió identificar a "49/UPEJOL/18" con la entidad "resolución":

{"source":"RESOLUCIÓN N.° 49/UPEJOL/18  ","etiqueta-boletin-3":{"annotations":{"labels":[{"label":"Date","shortDisplayName":"Dat"},{"label":"Resolution","shortDisplayName":"R"},{"label":"Type","shortDisplayName":"T"},{"label":"Motivo","shortDisplayName":"M"},{"label":"Awarded company","shortDisplayName":"A"},{"label":"Department","shortDisplayName":"I"},{"label":"Amount","shortDisplayName":"A"}],"entities":[{"label":"Resolution","startOffset":15,"endOffset":27}]}},"etiqueta-boletin-3-metadata":{"job-name":"labeling-job/etiqueta-boletin-3","type":"groundtruth/text-span","creation-date":"2023-10-16T11:28:08.355345","human-annotated":"yes","entities":[{"confidence":0}]}}



Como puede observarse, en primera instancia habíamos intentado procesar la data completa de nuestro boletín, pero el servicio reconoció 41.810 sentencias para etiquetar. De este modo, al achicar nuestra data de input, solo taggeamos 186.

<img src="img/label_job.png">

#### Etiquetar nuestra data

Dentro del entorno privado generado en primera instancia, realizaremos el trabajo de etiquetado de nuestra data.

<img src="img/etiquetar_nuestra_data.png">

En el ejemplo anterior, se observa cómo funciona el proceso de etiquetado y matcheo de frases y palabras dentro del corpus de texto con las entidades predefinidas. En este caso, asignamos a la empresa "POSTRES BALCARCE S.A." con una "awarded company", es decir, con una "empresa adjudicataria" en un proceso licitatorio.

Aquí mostramos cómo queda configurado el taggeo definitivo de nuestros datos:

<img src="img/cjto_datos_etiquetados.png">

Como fue mencionado, el template de CloudFormation configuró el bucket de S3 para correr una función Lambda cada vez que existan objetos nuevos dentro de la dirección manifests/output/output.manifest.

La función Lambda carga el archivo manifiesto aumentado y lo convierte en dos nuevos archivos (uno .csv y otro .txt).

El archivo .csv contiene las anotaciones específicas realizadas sobre la data y el .txt es el archivo puro que fue cargado para hacer el reconocimiento y taggeo de entidades.

### Entrenamiento del modelo NER

Finalmente, entrenaremos nuestro modelo con Amazon Comprehend.

Para poder hacerlo, Comprehend "exige" una serie de atributos en nuestra data. Uno de ellos es la longitud. Si bien nosotros habíamos definido 7 entidades (fecha, resolución, tipo de resolución, motivo de contratación, empresa adjudicataria, importe adjudicado y ministerio), en nuestro corpus de datos no habíamos podido definir la suficiente cantidad de veces cada una de ellas. Por ejemplo, la fecha había sido definida una sola vez (porque tomamos como input solo una resolución de las tantas que figuran dentro del boletín). En este sentido, Comprehend requiere la repetición de una entidad en al menos 25 oportunidades para poder efectuar el trabajo. Las dos únicas entidades que se repetían al menos 25 veces dentro de nuestro corpus de texto eran "awarded company" (empresa adjudicataria) y "motivo de contratación". Por esa razón, solo entrenamos nuestro modelo basándonos en estas entidades.

Para comenzar con el proceso de entrenamiento:

- En Comprehend creamos un trabajo de entrenamiento customizado, en el cual definimos las dos entidades antes mencionadas.
- Además, se definieron las carpetas dentro de nuestro bucket de S3 para tomar el archivo con las anotaciones y también los archivos que deberían considerarse como input/documentos de procesamiento.
- Se creó un rol específico para poder acceder a los buckets y poder realizar este ejercicio.
- Finalmente, se entrenó el modelo. El proceso demoró 10 minutos y obtuvo un F1-score de 70.96

<img src="img/cer.png">
<img src="img/cer2.png">
<img src="img/cer3.png">
<img src="img/cer4.png">
<img src="img/cer5.png">
<img src="img/cer6.png">

Terminado el entrenamiento, decidimos utilizar nuestro modelo para etiquetar data nueva, con la cual nunca habíamos interactuado. Es decir, probamos nuestro modelo.

Para ello, utilizamos como ejemplo al Boletín Oficial 5351, correspondiente al día 11 de abril de 2018.

Ejecutamos un proceso de análisis, donde definimos el bucket de S3 desde el cual tomar la data, también  en donde almacenar los resultados, generamos los roles IAM para poder llevar adelante el procedimiento y, por sobre todas las cosas, definimos el nombre del modelo que debía considerar para llevar adelante la tarea (en este caso, "my-ner-boletin-02").

<img src="img/analisis.png">
<img src="img/analisis2.png">

El job corrió durante 18 minutos y arrojó un archivo en formato .tar.gz que descomprimimos para acceder a un json, que exploraremos a continuación.

<img src="img/analisis3.png">
<img src="img/analisis4.png">

Este es el archivo final, que obtuvimos luego de descomprimir el .tar.gz que descargamos desde S3.

<img src="img/analisis5.png">

### Exploración sobre la data generada

Comenzamos por importar la librería de pandas y por leer nuestro archivo (que guardamos en formato json), con la respuesta de nuestro modelo. El mismo lo convertiremos en un dataframe de pandas.

In [3]:
import pandas as pd

df = pd.read_json('json/output.json', lines=True)

Analizamos las primeras dos filas de nuestro archivo. Como puede verse, hay muchas líneas de texto que no tienen una entidad reconocida, y por eso figuran con la primera columna vacía. Sin embargo, hay otras para las que sí logró detectar información.

La misma puede observarse en un formato poco amigable, por lo que iniciaremos un proceso de readaptación de la data para poder trabajar con ella de forma adecuada.

In [4]:
df.head(2)

,Entities,File,Line
0,[],BO-5351.txt,46
1,[],BO-5351.txt,47


In [5]:
# linea del archivo para la que sí encontró entidades

df.iloc[61]

Entities    [{'BeginOffset': 0, 'EndOffset': 10, 'Score': ...
File                                              BO-5351.txt
Line                                                      108
Name: 61, dtype: object

In [6]:
# En una lista almacenamos los DataFrames en forma aplanada
datos_aplanados = []

# Iteramos a través de las filas de nuestro DataFrame
for index, row in df.iterrows():
    if 'Entities' in row:
        aplanado = pd.json_normalize(row['Entities'])
        datos_aplanados.append(aplanado)

# Concatenamos los DataFrames aplanados en uno solo
df_2 = pd.concat(datos_aplanados, ignore_index=True)

# Inspeccionamos el DataFrame final (df_2)
print(df_2)

      BeginOffset  EndOffset     Score                             Text  \
0               0         10  0.571418                       Ministerio   
1              19         50  0.995079  JUEGOS OLÍMPICOS DE LA JUVENTUD   
2               0         10  0.728365                       Ministerio   
3               0         10  0.567420                       Ministerio   
4               0         10  0.782546                       Ministerio   
...           ...        ...       ...                              ...   
3732            1          7  0.842248                           603130   
3733           44         50  0.814382                           603210   
3734            0          7  0.876399                          (603310   
3735           99        129  0.913092   C I U D A D DE B U E N O S A I   
3736           15         41  0.951447       LOS JUEGOS OLÍMPICOS DE LA   

                 Type  
0     AWARDED COMPANY  
1              MOTIVO  
2     AWARDED COMPANY  
3  

Inspeccionamos las primeras 15 filas, para asegurarnos que las entidades de "AWARDED COMPANY" y "MOTIVO" hayan sido correctamente asignadas.

In [7]:
df_2.head(15)

,BeginOffset,EndOffset,Score,Text,Type
0,0,10,0.571418,Ministerio,AWARDED COMPANY
1,19,50,0.995079,JUEGOS OLÍMPICOS DE LA JUVENTUD,MOTIVO
2,0,10,0.728365,Ministerio,AWARDED COMPANY
3,0,10,0.567420,Ministerio,AWARDED COMPANY
4,0,10,0.782546,Ministerio,AWARDED COMPANY
5,0,10,0.878687,Ministerio,AWARDED COMPANY
6,27,78,0.912655,Servicios Integrales de Alimentación SA - Arki...,AWARDED COMPANY
7,81,84,0.859064,UTE,AWARDED COMPANY
8,27,55,0.938471,Comahue Seguridad Privada SA,AWARDED COMPANY
9,27,63,0.862666,Líderes Consultores de Seguridad SRL,AWARDED COMPANY


A simple vista, podemos observar que el modelo reconoció entidades de forma incorrecta, mayormente. En muchos casos, la palabra "Ministerio" la reconoció como una empresa adjudicataria, cuando en realidad siquiera debería haber sido considerada. Como explicaremos en la conclusión, consideramos que esto se debe a la poca cantidad de data utilizada para entrenar nuestro modelo.

De todos modos, continuemos explorando este nuevo DataFrame. Realizaremos un agrupamiento de nuestra data por la columna "Type".

In [8]:
df_2.groupby('Type')[['Text']].count().sort_values(by='Text', ascending=False)

,Text
Type,
AWARDED COMPANY,2402
MOTIVO,1335


Luego generaremos una nueva columna de forma de agrupar solo aquellos reconocimientos con un score mayor al 90%.

In [9]:
for index, row in df_2.iterrows():
    if row['Score'] > 0.90:
        df_2.at[index, 'Agrup'] = 'Mayor al 90%'
    else:
        df_2.at[index, 'Agrup'] = 'Menor al 90%'

In [10]:
df_3 = df_2[df_2['Agrup'] == 'Mayor al 90%']

Agrupamos nuestra data primero por texto (o la entidad en sí misma) y luego por tipo de entidad y entidad, contando la cantidad de veces que aparecen en nuestro DataFrame.

In [11]:
df_3.groupby('Text')[['Type']].count().sort_values(by='Type', ascending=False).head(10)

,Type
Text,
RESOLUCIÓN N.°,93
RESOLUCIÓN N.º,66
www.buenosairescompras.gob.ar,49
DISPOSICIÓN N.°,38
MINISTERIO DE SALUD,37
ANEXOQue,33
Mantelectric ICISA,32
SECRETARIO DE TRANSPORTE,29
DE LA CIUDAD AUTÓNOMA DE BUENOS AIRES,24


In [12]:
df_3.groupby(['Type','Text'])[['Score']].count().sort_values(by='Score', ascending=False).head(10)

Score
Type            Text                                        
MOTIVO          RESOLUCIÓN N.°                            93
                RESOLUCIÓN N.º                            66
AWARDED COMPANY www.buenosairescompras.gob.ar             49
MOTIVO          DISPOSICIÓN N.°                           38
                MINISTERIO DE SALUD                       37
AWARDED COMPANY ANEXOQue                                  33
                Mantelectric ICISA                        32
MOTIVO          SECRETARIO DE TRANSPORTE                  29
                DE LA CIUDAD AUTÓNOMA DE BUENOS AIRES     24
                DEL ENTE AUTÁRQUICO TEATRO COLÓN          22

Puede verse que el modelo ha reconocido las entidades de forma incorrecta, en mayor medida. Para las 10 entidades con más apariciones en nuestro DataFrame, solo una "Mantelectric ICISA" fue reconocida de forma adecuada.

Sin embargo, probablemente el modelo esté reconociendo cosas que no deberían reconocerse (lo cual es un problema), pero intentaremos certificar que efectivamente esté reconociendo aquellas entidades que sí deberían tenerse en cuenta.

Para ello compararemos la extracción manual que en su momento realizamos durante nuestra investigación sobre los Juegos Olímpicos de la Juventud 2018 (YOG 2018). Si bien este DataFrame estará supeditado a aquellas cuestiones vinculadas de forma estricta con este megaevento deportivo, consideramos que es un buen parámetro para entender la capacidad de nuestro modelo en reconocer entidades dentro del Boletín Oficial. 

Importamos nuestro archivo con la data analizada durante el proyecto.

In [13]:
yog = pd.read_csv("Consolidado YOG 2012-19.csv")
yog.head(2)

,Fecha,Nro_expediente,Nro_BO,Fecha_BO,Organismo,Descripción,Empresa,Monto_ARS,Monto_USD
0,28/06/13,707.752/13,4429,02/07/14,Ministerio de Desarrollo Económico,Puesta en Valor del Polideportivo del Barrio L...,COOPERATIVA DE TRABAJO LA UNION LIMITADA,"$ 250,000.00","USD 30,637.25"
1,12/08/13,3.511.323/13,4429,02/07/14,Ministerio de Desarrollo Económico,Puesta en Valor del Polideportivo del Barrio L...,COOPERATIVA DE TRABAJO LA UNION LIMITADA,"$ 150,000.00","USD 18,382.35"


Filtramos todas las anotaciones realizadas para el Boletín Oficial número 5351, del 11 de abril de 2018.

In [14]:
yog.loc[yog['Nro_BO'] == 5351]

,Fecha,Nro_expediente,Nro_BO,Fecha_BO,Organismo,Descripción,Empresa,Monto_ARS,Monto_USD
10,11/04/18,04.608.204- MGEYA-UPEJOL/18,5351,11/04/18,"Ministerio de Modernización, Innovación y Tecn...",Contratación de Patrocinio Local para los Jueg...,"PISTRELLI, HENRY MARTIN Y ASOCIADOS SRL","$ 4,145,280.00","USD 203,200.00"
242,27/03/18,EX-2018-02461482-MGEYA-IVC,5351,11/04/18,Instituto de la Vivienda de la Ciudad de Bueno...,"Contratación de Saneamiento, Mantenimiento y R...",COOPERATIVA DE TRABAJO ECOLOGICA ARGENTINA LTDA,"$ 6,037,751.50","USD 295,968.21"


Puede verse que hay solo dos filas con información respectiva a los YOG 2018 dentro de ese boletín. En nuestra data no hemos siquiera podido reconocer el importe, pero sí el nombre de la empresa. Intentaremos buscar en nuestra data si encontró a estas dos empresas dentro de las más de 3.000 filas. 

In [15]:
#df_2[(df_2['Text'] == 'Pistrelli') | (df_2['Text'] == 'Ecologica Argentina')]

In [16]:
df_2[(df_2['Text'].str.contains('Pistrelli', case=False)) | (df_2['Text'].str.contains('Ecologica', case=False))]

,BeginOffset,EndOffset,Score,Text,Type,Agrup
2262,0,19,0.955421,Ecologica Argentina,AWARDED COMPANY,Mayor al 90%
2991,21,30,0.925592,Pistrelli,AWARDED COMPANY,Mayor al 90%


Se puede observar que las dos entidades fueron reconocidas como compañías adjudicatarias, pero en ambos casos el nombre figura recortado. Si bien la asignación con el tipo de entidad fue la adecuada, deberemos inspeccionar aún más para hacer de este modelo una herramienta de detección confiable.

### Conclusiones y comentarios de cara al trabajo de tesis:

<ol>
    <li>Incorporar dentro del moldeo de la data de input la posibilidad de trabajar con un archivo de texto sin saltos de línea, ya que en el ejercicio de etiquetado de la data se ha observado que muchas veces las oraciones llegan partidas y eso dificulta la asociación con las entidades.</li>
    <li>Lógicamente, utilizar corpus de texto más extensos, a los fines de mejorar el ejercicio de predicción de nuestro modelo. En este caso, hemos decidido hacerlo con un archivo más pequeño debido a la falta de tiempo en la implementación y a que el objetivo principal era dejar operativo el modelo.</li>
    <li>Implementar otro tipo de approachs frente al mismo problema, por ejemplo, redes neuronales con transformadores. Hay soluciones que pueden ser de soporte, como <a href="https://www.github.com/AymurAI/dev.git"> AymurAI</a> o <a href="https://github.com/MuckRock/sidekick"> Sidekick</a>.</li>
    <li> Se podría inspeccionar, también, en la creación de una base SQL dentro de AWS, como podría ser  Amazon RDS, para realizar analítica y Quicksight, para almacenar las métricas y generar visualizaciones sobre los datos más importantes. </li>